In [1]:
!conda env list


# conda environments:
#
base                   /opt/conda
adsp-genai           * /opt/conda/envs/adsp-genai
interview-ai           /opt/conda/envs/interview-ai
jupyterlab             /opt/conda/envs/jupyterlab
jupyterlab4-preview    /opt/conda/envs/jupyterlab4-preview
pytorch                /opt/conda/envs/pytorch
tensorflow             /opt/conda/envs/tensorflow



In [2]:
# !pip install beautifulsoup4
# !pip install requests
# !pip install --upgrade PyMuPDF

import pandas as pd
import requests
from bs4 import BeautifulSoup
import fitz  # PyMuPDF

In [3]:
df = pd.read_csv("../Data/Dataset - RAG - V2.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Category     25 non-null     object 
 1   Main Pages   29 non-null     object 
 2   Sub Pages    24 non-null     object 
 3   Description  3 non-null      object 
 4   Link         37 non-null     object 
 5   Notes        0 non-null      float64
 6   Check        37 non-null     bool   
 7   PIC          37 non-null     object 
dtypes: bool(1), float64(1), object(6)
memory usage: 2.2+ KB


In [5]:
# def scrape_page_content(url):
#     try:
#         resp = requests.get(url, timeout=15)
#         resp.raise_for_status()
#     except requests.RequestException as e:
#         return f"Error: {e}"

#     soup = BeautifulSoup(resp.text, "html.parser")

#     for tag in soup.find_all(['header', 'footer', 'nav', 'aside']):
#         tag.decompose()

#     main_content = (
#         soup.find("div", class_="entry-content") or
#         soup.find("main") or
#         soup.find("article")
#     )

#     if not main_content:
#         divs = soup.find_all("div")
#         main_content = max(divs, key=lambda d: len(d.get_text()), default=soup)

#     return main_content.get_text(separator="\n", strip=True)

# def extract_pdf_text_from_url(pdf_url):
#     try:
#         response = requests.get(pdf_url, timeout=15)
#         if response.status_code == 200:
#             with open("temp.pdf", "wb") as f:
#                 f.write(response.content)
#             with fitz.open("temp.pdf") as doc:
#                 text = "\n".join([page.get_text() for page in doc])
#             return text.strip() if text else "Empty PDF"
#         else:
#             return f"Failed to fetch PDF: HTTP {response.status_code}"
#     except Exception as e:
#         return f"PDF Error: {e}"

In [6]:
# scraped_contents = []

# for url in df["Link"]:
#     if pd.isna(url):
#         scraped_contents.append("No link")
#     elif url.strip().lower().endswith(".pdf"):
#         scraped_contents.append(extract_pdf_text_from_url(url.strip()))
#     else:
#         scraped_contents.append(scrape_page_content(url.strip()))

# df["scraped_content"] = scraped_contents
# # df.to_csv("scraped_output.csv", index=False)

In [7]:
pd.set_option("display.max_colwidth", 500)
# pd.DataFrame(scraped_contents)

In [19]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
from typing import List, Dict, Any
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import re


# def scrape_sections_with_metadata(url: str) -> List[Dict[str, Any]]:
#     resp = requests.get(url, timeout=15)
#     resp.raise_for_status()
#     soup = BeautifulSoup(resp.text, "html.parser")
#     # remove boilerplate
#     for tag in soup.find_all(['header','footer','nav','aside']):
#         tag.decompose()

#     # find the “main” container
#     main = (
#         soup.find("div", class_="entry-content")
#         or soup.find("main")
#         or soup.find("article")
#     )
#     if not main:
#         divs = soup.find_all("div")
#         main = max(divs, key=lambda d: len(d.get_text()), default=soup)

#     page_title = soup.title.string.strip() if soup.title else None
#     timestamp  = datetime.utcnow().isoformat()

#     segments: List[Dict[str, Any]] = []

#     # iterate through every <p> and <li> under main_content
#     for tag in main.find_all(['p','li']):
#         text = tag.get_text(strip=True)
#         if not text:
#             continue
#         # find the nearest preceding h2 or h3
#         heading = tag.find_previous(['h2','h3'])
#         section = heading.get_text(strip=True) if heading else page_title or "General"

#         segments.append({
#             "text": text,
#             "metadata": {
#                 "source_url":   url,
#                 "retrieved_at": timestamp,
#                 "title":        page_title,
#                 "section":      section
#             }
#         })

#     return segments

import requests
from bs4 import BeautifulSoup
from datetime import datetime
from typing import List, Dict, Any

def scrape_sections_with_metadata(url: str) -> List[Dict[str, Any]]:
    try:
        resp = requests.get(url, timeout=15)
        resp.raise_for_status()
    except requests.RequestException as e:
        print(f"[Request Error] {url} — {e}")
        return []

    try:
        soup = BeautifulSoup(resp.text, "html.parser")
    except Exception as e:
        print(f"[HTML Parsing Error] {url} — {e}")
        return []

    # Remove page boilerplate and sidebar
    try:
        for tag in soup.find_all(['header', 'footer', 'nav', 'aside']):
            tag.decompose()
        sidebar_nav = soup.find("ul", id="menu-sidebar-menu")
        if sidebar_nav:
            sidebar_nav.decompose()
    except Exception as e:
        print(f"[Cleanup Error] {url} — {e}")

    # Find main content
    main = (
        soup.find("div", class_="entry-content")
        or soup.find("main")
        or soup.find("article")
    ) or soup

    page_title = (soup.title.string or "").strip() if soup.title else "Unknown"
    timestamp = datetime.utcnow().isoformat()
    segments: List[Dict[str, Any]] = []

    # === 1. ACCORDION CONTENT ===
    for content_div in soup.find_all("div", class_="accordion-content"):
        try:
            text = content_div.get_text(strip=True)
            if not text:
                continue

            section = None
            super_section = None

            section_anchor = content_div.find_previous("a", class_="accordion-title")
            if section_anchor:
                section = section_anchor.get_text(strip=True)

            panel = content_div.find_parent("div", class_="tabs-panel")
            if panel and panel.has_attr("aria-labelledby"):
                label_id = panel["aria-labelledby"]
                label_tag = soup.find("a", id=label_id)
                if label_tag:
                    super_section = label_tag.get_text(strip=True)

            segments.append({
                "text": text,
                "metadata": {
                    "source_url": url,
                    "retrieved_at": timestamp,
                    "title": page_title,
                    "section": section or "Unknown Section",
                    "super_section": super_section or "Unknown Super Section"
                }
            })
        except Exception as e:
            print(f"[Accordion Parse Error] {url} — {e}")
            continue

    # === 2. GENERAL <p> and <li> TAGS ===
    for tag in main.find_all(['p', 'li']):
        try:
            if tag.name == 'li' and tag.get('role') == 'presentation':
                continue
            if tag.has_attr('data-accordion-item'):
                continue

            text = tag.get_text(strip=True)
            if not text:
                continue

            section = None
            super_section = None
            h_tag = None

            panel = tag.find_parent("div", class_="tabs-panel")
            if panel and panel.has_attr("aria-labelledby"):
                label_id = panel["aria-labelledby"]
                label_tag = soup.find("a", id=label_id)
                if label_tag:
                    super_section = label_tag.get_text(strip=True)

            acc = tag.find_previous("a", class_="accordion-title")
            if acc:
                section = acc.get_text(strip=True)

            wrapper = tag.find_parent("div", class_="list-accordion")
            if wrapper:
                active = wrapper.select_one("li.tabs-title.is-active a")
                if active:
                    section = active.get_text(strip=True)
                    if not super_section:
                        super_section = section

            if not section:
                tab_li = tag.find_previous("li", class_="tabs-title")
                if tab_li:
                    a = tab_li.find("a")
                    if a:
                        section = a.get_text(strip=True)
                        if not super_section:
                            super_section = section

            if not section:
                h_tag = tag.find_previous(["h2", "h3"])
                if h_tag:
                    section = h_tag.get_text(strip=True)

            if not section:
                section = page_title or "General"

            if not super_section and h_tag:
                if h_tag.name == "h3":
                    h2_above = h_tag.find_previous("h2")
                    if h2_above:
                        super_section = h2_above.get_text(strip=True)
                elif h_tag.name == "h2":
                    h1_above = h_tag.find_previous("h1")
                    if h1_above:
                        super_section = h1_above.get_text(strip=True)

            segments.append({
                "text": text,
                "metadata": {
                    "source_url": url,
                    "retrieved_at": timestamp,
                    "title": page_title,
                    "section": section,
                    "super_section": super_section or None
                }
            })
        except Exception as e:
            print(f"[Tag Parse Error] {url} — {e}")
            continue

    return segments

import os
import requests
import fitz  # PyMuPDF
from datetime import datetime
from typing import Dict, Any

def extract_pdf_text_from_url(pdf_url: str) -> Dict[str, Any]:
    """
    Fetches a PDF from `pdf_url`, extracts its text, and returns a dict:
      {
        "text": <full text of the PDF>,
        "metadata": {
          "source_url":   <pdf_url>,
          "retrieved_at": <UTC ISO timestamp>,
          "title":        <PDF metadata title or filename fallback>,
          "section":      <same as title>,
          "super_section": None
        }
      }
    """
    # Fetch
    try:
        resp = requests.get(pdf_url, timeout=15)
        resp.raise_for_status()
    except Exception as e:
        return {
            "text": "",
            "metadata": {
                "source_url":   pdf_url,
                "retrieved_at": datetime.utcnow().isoformat(),
                "title":        None,
                "section":      None,
                "super_section": None
            }
        }

    # Save locally
    temp_path = "temp.pdf"
    with open(temp_path, "wb") as f:
        f.write(resp.content)

    # Extract text and PDF's own metadata
    try:
        with fitz.open(temp_path) as doc:
            # join all page texts
            full_text = "\n".join(page.get_text() for page in doc).strip() or "Empty PDF"
            pdf_meta = doc.metadata or {}
    except Exception:
        full_text = ""
        pdf_meta = {}

    # Clean up
    try:
        os.remove(temp_path)
    except OSError:
        pass

    # Determine a sensible title/section
    filename = os.path.basename(pdf_url).split("?")[0]
    title = pdf_meta.get("title") or filename or "PDF Document"

    metadata = {
        "source_url":    pdf_url,
        "retrieved_at":  datetime.utcnow().isoformat(),
        "title":         title,
        "section":       title,
        "super_section": None
    }

    return {
        "text":     full_text,
        "metadata": metadata
    }



# 2) CHUNKER: break a string into overlapping chunks
def chunk_text(
    text: str,
    chunk_size: int = 800,
    chunk_overlap: int = 80
) -> List[str]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " "]
    )
    return splitter.split_text(text)

# 3) PIPELINE: from df of URLs → df of chunks + metadata
def build_chunks_dataframe(
    df: pd.DataFrame,
    url_col: str = "url",
    chunk_size: int = 800,
    chunk_overlap: int = 80
) -> pd.DataFrame:
    all_chunks, all_metas = [], []

    for idx, row in df.iterrows():
        url = row[url_col]
        try:
            segments = scrape_sections_with_metadata(url)
        except Exception as e:
            print(f"⚠️  Skipping {url} due to scrape error: {e}")
            continue

        for seg in segments:
            seg_chunks = chunk_text(seg["text"], chunk_size, chunk_overlap)
            all_chunks.extend(seg_chunks)
            for _ in seg_chunks:
                all_metas.append({
                    **seg["metadata"],
                    "row_index": idx
                })

    return pd.DataFrame({
        "chunk": all_chunks,
        "metadata": all_metas
    })


# 4) VECTOR STORE: embed & persist
def build_and_persist_chroma(
    df_chunks: pd.DataFrame,
    embed_model: str = "sentence-transformers/all-MiniLM-L6-v2",
    persist_dir: str = "./chroma_db"
) -> Chroma:
    # prepare texts & metas
    texts = df_chunks["chunk"].tolist()
    metas = df_chunks["metadata"].tolist()

    # embedder
    embedder = HuggingFaceEmbeddings(
        model_name=embed_model,
        encode_kwargs={"normalize_embeddings": True}
    )
    # build and persist
    db = Chroma.from_texts(
        texts=texts,
        embedding=embedder,
        metadatas=metas,
        persist_directory=persist_dir
    )
    db.persist()
    return db

# === USAGE ===
# assume you have a DataFrame `df_urls` with a column "url"
# Step A: scrape + chunk + metadata → df_chunks
# df_chunks = build_chunks_dataframe(df, url_col="Link")

# Step B: build & persist your Chroma vector store
# db = build_and_persist_chroma(df_chunks, embed_model="sentence-transformers/all-MiniLM-L6-v2")


In [57]:
scrape_sections_with_metadata("https://datascience.uchicago.edu/education/masters-programs/in-person-program/")
# scrape_sections_with_metadata("https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/how-to-apply/")
# scrape_sections_with_metadata("https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/")
# extract_pdf_text_from_url("https://datascience.uchicago.edu/wp-content/uploads/2024/11/MS-ADS-Capstone-Sponsor-Guide-2025-2.pdf")

[{'text': 'Career Seminar (Seminar, required)The Pass/Fail Career Seminar supports the development of industry professional skills, job and/or internship searches, and other in-demand areas of competency among today’s employers. Students enroll in the Career Seminar each quarter in order to engage in unique content throughout their degree program. Students with significant full-time work experience may be eligible to waive this course. 0 units, no cost.Introduction to Statistical Concepts (Foundational, optional)This course is held in the five weeks leading up to the start of your first quarter and provides general exposure to basic statistical concepts necessary for success in advanced courses in the program. 0 units, no cost.R for Data Science (Foundational, optional)This course is held the five weeks leading up to the start of your first quarter and is an introduction to the essential concepts and techniques for the statistical computing language R. 0 units, no cost.Python for Data 

In [11]:
# pd.set_option("display.max_rows", None)
# df_chunks.iloc[53:70,:]

In [13]:
df_filtered = df.drop(index = [4, 5, 14, 15, 24, 25])

In [16]:
# scraped_output = df_filtered['Link'].apply(scrape_sections_with_metadata)

In [42]:
scraped_output = []

for url in df_filtered["Link"]:
    if pd.isna(url) or not str(url).strip():
        # no segments for missing links
        page_segments = []
    else:
        u = url.strip()
        if u.lower().endswith(".pdf"):
            # extractor returns a single dict → wrap in a list
            pdf_seg = extract_pdf_text_from_url(u)
            page_segments = [pdf_seg] if isinstance(pdf_seg, dict) else []
        else:
            # returns a list of dicts already
            page_segments = scrape_sections_with_metadata(u)

    scraped_output.append(page_segments)


In [38]:
# for page_segments in scraped_output:
#     for segment in page_segments:
#         print(segment)

In [34]:
# pd.DataFrame(flat_segments)

In [43]:
# Flatten nested lists into one list of dicts
flat_segments = [segment
                 for page_segments in scraped_output
                 for segment in page_segments]

# Now build DataFrame; it will infer two columns: text and metadata
df_scraped = pd.DataFrame(flat_segments, columns=["text","metadata"])

df_scraped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1304 entries, 0 to 1303
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      1304 non-null   object
 1   metadata  1304 non-null   object
dtypes: object(2)
memory usage: 20.5+ KB


In [44]:
len(flat_segments)

1304

In [66]:
# df_scraped = pd.DataFrame([segment for segment in scraped_output][0])

# df_scraped.head()

In [49]:
df_scraped[df_scraped['metadata'].apply(lambda x: x.get('section', '')) == 'MS-ADS Capstone Sponsor Guide 2025']

,text,metadata
70,"The University of Chicago Master's in Applied Data Science program is a dynamic hub for aspiring leaders\n and analytical thinkers. The curriculum integrates business strategy with immersive project-based learning\n. Our students go beyond mastering data science, they apply it in real-world scenarios across different sectors. \n01 | 2025\nMS in Applied Data Science\nCapstone Sponsor Guidance 2025\nThe University of Chicago\nStudents in the \nMS-ADS program\nhave completed\nover\n300 Capstone...","{'source_url': 'https://datascience.uchicago.edu/wp-content/uploads/2024/11/MS-ADS-Capstone-Sponsor-Guide-2025-2.pdf', 'retrieved_at': '2025-07-22T20:17:29.910091', 'title': 'MS-ADS Capstone Sponsor Guide 2025', 'section': 'MS-ADS Capstone Sponsor Guide 2025', 'super_section': None}"


In [45]:
df_scraped.tail(5)

,text,metadata
1299,"An MBA from Booth is a significant investment in yourself, your intellect, your relationships, and your future. Here's how to finance it.","{'source_url': 'https://www.chicagobooth.edu/mba/full-time/admissions/cost', 'retrieved_at': '2025-07-22T20:17:38.561681', 'title': 'Full-Time MBA Cost | The University of Chicago Booth School of Business', 'section': 'Explore Scholarships and Financial Aid', 'super_section': 'Full-Time MBA Cost'}"
1300,The first step in joining our transformational Full-Time MBA Program is to apply for admission.,"{'source_url': 'https://www.chicagobooth.edu/mba/full-time/admissions/cost', 'retrieved_at': '2025-07-22T20:17:38.561681', 'title': 'Full-Time MBA Cost | The University of Chicago Booth School of Business', 'section': 'Begin Your MBA Journey', 'super_section': 'Full-Time MBA Cost'}"
1301,Start Your Application,"{'source_url': 'https://www.chicagobooth.edu/mba/full-time/admissions/cost', 'retrieved_at': '2025-07-22T20:17:38.561681', 'title': 'Full-Time MBA Cost | The University of Chicago Booth School of Business', 'section': 'Begin Your MBA Journey', 'super_section': 'Full-Time MBA Cost'}"
1302,Do you have questions about the Full-Time MBA Program cost and financial aid? Contact us. We look forward to hearing from you!,"{'source_url': 'https://www.chicagobooth.edu/mba/full-time/admissions/cost', 'retrieved_at': '2025-07-22T20:17:38.561681', 'title': 'Full-Time MBA Cost | The University of Chicago Booth School of Business', 'section': 'Contact Us', 'super_section': 'Full-Time MBA Cost'}"
1303,Financial AidEmail Us,"{'source_url': 'https://www.chicagobooth.edu/mba/full-time/admissions/cost', 'retrieved_at': '2025-07-22T20:17:38.561681', 'title': 'Full-Time MBA Cost | The University of Chicago Booth School of Business', 'section': 'Contact Us', 'super_section': 'Full-Time MBA Cost'}"


In [50]:
df_scraped.to_csv("scraped_output_metadata.csv")

In [59]:
df = pd.read_csv("../Data/scraped_output.csv")

In [60]:
df

,Category,Main Pages,Sub Pages,Description,Link,Notes,Check,PIC,scraped_Content
0,In Person,Main Page,NaN,NaN,https://datascience.uchicago.edu/education/masters-programs/in-person-program/,NaN,True,Andrew,"MS in Applied Data Science\nfacet-arrow-down\nOverview\nIn-Person Program\nOnline Program\nCapstone Projects\nCourse Progressions\nHow to Apply\nEvents & Deadlines\nTuition, Fees, & Aid\nOur Students\nFaculty, Instructors, Staff\nFAQs\nExplore the MS-ADS Campus\nCareer Outcomes\nGet In Touch\nYour Career Success\nTake the next step to advance your career with UChicago’s MS in Applied Data Science.\nThe In-Person program admits full- and part-time students for entrance in Autumn quarter annua..."
1,Applied Data Science,Capstone,NaN,NaN,https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/,NaN,True,Andrew,"MS in Applied Data Science\nfacet-arrow-down\nOverview\nIn-Person Program\nOnline Program\nCapstone Projects\nCourse Progressions\nHow to Apply\nEvents & Deadlines\nTuition, Fees, & Aid\nOur Students\nFaculty, Instructors, Staff\nFAQs\nExplore the MS-ADS Campus\nCareer Outcomes\nGet In Touch\nThe culminating experience in the\nMaster’s in Applied Data Science\nprogram is a\nCapstone Project where you’ll put your knowledge and skills into practice\n. You will immerse yourself in a real busine..."
2,Applied Data Science,Capstone,Capstone Guide,NaN,https://datascience.uchicago.edu/wp-content/uploads/2024/11/MS-ADS-Capstone-Sponsor-Guide-2025-2.pdf,NaN,False,Andrew,"The University of Chicago Master's in Applied Data Science program is a dynamic hub for aspiring leaders\n and analytical thinkers. The curriculum integrates business strategy with immersive project-based learning\n. Our students go beyond mastering data science, they apply it in real-world scenarios across different sectors. \n01 | 2025\nMS in Applied Data Science\nCapstone Sponsor Guidance 2025\nThe University of Chicago\nStudents in the \nMS-ADS program\nhave completed\nover\n300 Capstone..."
3,Applied Data Science,Course Progression,NaN,Full Time and Part Time Progression,https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/course-progressions/,NaN,False,Andrew,"MS in Applied Data Science\nfacet-arrow-down\nOverview\nIn-Person Program\nOnline Program\nCapstone Projects\nCourse Progressions\nHow to Apply\nEvents & Deadlines\nTuition, Fees, & Aid\nOur Students\nFaculty, Instructors, Staff\nFAQs\nExplore the MS-ADS Campus\nCareer Outcomes\nGet In Touch\nFlexible Formats\nWe prepare you to advance in the\ncompetitive landscape of data science career paths with a focus on industry applications\n. Full- and Part-time options are available in both the\nOnl..."
4,Applied Data Science,Course Progression,Online Program,NaN,https://datascience.uchicago.edu/education/masters-programs/online-program/,NaN,False,Andrew,"MS in Applied Data Science\nfacet-arrow-down\nOverview\nIn-Person Program\nOnline Program\nCapstone Projects\nCourse Progressions\nHow to Apply\nEvents & Deadlines\nTuition, Fees, & Aid\nOur Students\nFaculty, Instructors, Staff\nFAQs\nExplore the MS-ADS Campus\nCareer Outcomes\nGet In Touch\nRigor Meets Flexibility\nYou will benefit from\nthe same top faculty, bright peer group, and academic rigor\nas the In-Person program. You may complete the program full- or part-time. Full-time and part..."
5,Applied Data Science,Course Progression,Offline Program,NaN,https://datascience.uchicago.edu/education/masters-programs/in-person-program/,NaN,False,Andrew,"MS in Applied Data Science\nfacet-arrow-down\nOverview\nIn-Person Program\nOnline Program\nCapstone Projects\nCourse Progressions\nHow to Apply\nEvents & Deadlines\nTuition, Fees, & Aid\nOur Students\nFaculty, Instructors, Staff\nFAQs\nExplore the MS-ADS Campus\nCareer Outcomes\nGet In Touch\nYour Career Success\nTake the next step to advance your career with UChicago’s MS in Applied Data Science.\nThe In-Person program admits ful

In [9]:
pd.set_option("display.max_colwidth", 1000000)
df.info()
df.iloc[2, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Category         25 non-null     object 
 1   Main Pages       29 non-null     object 
 2   Sub Pages        24 non-null     object 
 3   Description      3 non-null      object 
 4   Link             37 non-null     object 
 5   Notes            0 non-null      float64
 6   Check            37 non-null     bool   
 7   PIC              37 non-null     object 
 8   scraped_Content  37 non-null     object 
dtypes: bool(1), float64(1), object(7)
memory usage: 2.5+ KB


Category                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [110]:
df.iloc[18:20, :].to_csv("scraped_output_v2.csv", index=False)